In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import ToMarkdownLoader

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownTextSplitter

from langchain_community.document_loaders import UnstructuredMarkdownLoader

/Users/sahilsharma/miniconda3/envs/rag_qa/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [2]:
loaders = [
            UnstructuredMarkdownLoader("malware_xsoar.md"),
            UnstructuredMarkdownLoader("phishing_xsoar.md"),
        ]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
docs[0].page_content[:150]

'Malware: Customer Conversations\n\nTo help scale and automate investigations like this, we at Cortex XSOAR built the Malware Investigation and Response '

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

splits = text_splitter.split_documents(docs)

In [7]:
len(splits)

37

In [8]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/recursive'

In [9]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [10]:
print(vectordb._collection.count())

74


In [11]:
question = "how do you analyse the message in a phishing mail"


In [12]:
docs_result = vectordb.similarity_search(question,k=3)
len(docs_result)

3

In [13]:
docs_result

[Document(page_content="When a keyword is found in the email body, it will be displayed in the layout “Investigation” tab in a tag field, under the “Spear Phishing Investigation” section.\n\nSince it's a tag type field, the analyst can search and filter incidents and identify phishing mails containing the same keywords.\n\nDetermine phishing mail sender verdict as malicious\n\nOnce a phishing incident verdict is determined as malicious, the email sender address verdict will be set as malicious as well.", metadata={'source': 'phishing_xsoar.md'}),
 Document(page_content="When a keyword is found in the email body, it will be displayed in the layout “Investigation” tab in a tag field, under the “Spear Phishing Investigation” section.\n\nSince it's a tag type field, the analyst can search and filter incidents and identify phishing mails containing the same keywords.\n\nDetermine phishing mail sender verdict as malicious\n\nOnce a phishing incident verdict is determined as malicious, the em

In [14]:
llm_name = "gpt-3.5-turbo"

In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [16]:
question = "Can you summarise the phishing playbook?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [17]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/Users/sahilsharma/miniconda3/envs/rag_qa/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
from langchain.chains import RetrievalQA

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),return_source_documents=True
)

In [20]:
result = qa_chain({"query": question})

/Users/sahilsharma/miniconda3/envs/rag_qa/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [21]:
result["result"]

'The phishing playbook in the Cortex Marketplace content pack includes features for responding to phishing campaigns in 2023. It emphasizes immediate action upon detecting phishing attempts to mitigate harm by blocking associated indicators. The playbook includes a sub-playbook called "Spear Phishing Investigation" where analysts can input keywords related to the organization to detect their presence in phishing emails, helping to identify potential threats.'

In [22]:
result

{'query': 'Can you summarise the phishing playbook?',
 'result': 'The phishing playbook in the Cortex Marketplace content pack includes features for responding to phishing campaigns in 2023. It emphasizes immediate action upon detecting phishing attempts to mitigate harm by blocking associated indicators. The playbook includes a sub-playbook called "Spear Phishing Investigation" where analysts can input keywords related to the organization to detect their presence in phishing emails, helping to identify potential threats.',
 'source_documents': [Document(page_content="We’ve added new features to our most popular Cortex Marketplace content pack to help you better respond to phishing campaigns in 2023.\n\nPhishing - Generic v3 Playbook\n\nSemi-Automated Remediation\n\nWhen detecting phishing attempts, immediate action is essential to mitigate potential harm. By blocking indicators associated with the phishing attempt, we limit the attackers' ability to execute their malicious operations 